# Retail Orders Mini ETL

What it does:
1) Extract: Load CSV from /content/input/orders.csv OR generate synthetic data <BR>
2) Transform: Clean columns, types, compute total_price, deduplicate, filter bad rows<BR>
3) Load: Save to SQLite (tables: orders_clean, daily_revenue)<BR>
4) Verify: Quick SQL checks<BR>
5) Save artifacts: clean CSVs and the SQLite DB file

In [ ]:
# Install Statements----------------------
import os
import sqlite3
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
# Drive Path & Directories--------------------
BASE_DIR = Path("/content")
INPUT_DIR = BASE_DIR / "input"
OUTPUT_DIR = BASE_DIR / "outputs"
DB_PATH = OUTPUT_DIR / "retail_etl.db"
RAW_CSV = INPUT_DIR / "orders.csv"
USE_SYNTHETIC_IF_MISSING = True
SEED = 42

In [ ]:
# Ensure folders
INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# Helper: Generate synthetic sample--------------------

def generate_sample_orders(n=200, seed=SEED) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    categories = ["Grocery", "Electronics", "Clothing", "Stationery"]
    cities = ["Bengaluru", "Hyderabad", "Chennai", "Mumbai", "Pune"]
    start = pd.Timestamp("2024-07-01")
    dates = start + pd.to_timedelta(rng.integers(0, 60, size=n), unit="D")

    df = pd.DataFrame({
        "order_id": np.arange(1001, 1001+n),
        "order_date": dates.strftime("%Y-%m-%d"),
        "customer_id": rng.integers(1, 60, size=n),
        "product_id": rng.integers(101, 160, size=n),
        "category": rng.choice(categories, size=n),
        "city": rng.choice(cities, size=n),
        "qty": rng.integers(1, 6, size=n),
        "unit_price": np.round(rng.uniform(50, 2000, size=n), 2),
    })
    # Sprinkle a few issues (nulls/dups)
    df.loc[rng.choice(df.index, 5, replace=False), "city"] = None
    df = pd.concat([df, df.sample(3, random_state=seed)], ignore_index=True)  # add duplicates
    return df


## Extract

In [ ]:
if RAW_CSV.exists():
    print(f" Found input file: {RAW_CSV}")
    orders = pd.read_csv(RAW_CSV)
else:
    if USE_SYNTHETIC_IF_MISSING:
        print(" No input file found. Generating synthetic dataset...")
        orders = generate_sample_orders()
        # Save the raw synthetic for reference
        synthetic_path = OUTPUT_DIR / "orders_raw_synthetic.csv"
        orders.to_csv(synthetic_path, index=False)
        print(f" Synthetic raw saved at: {synthetic_path}")
    else:
        raise FileNotFoundError("No input CSV found and synthetic generation disabled.")

print("Raw shape:", orders.shape)

 No input file found. Generating synthetic dataset...
 Synthetic raw saved at: /content/outputs/orders_raw_synthetic.csv
Raw shape: (203, 8)


## Transform

In [ ]:
# make a copy first
df = orders.copy()

### Standardize column names (lowercase & underscores)

In [ ]:
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(" ", "_")
)

In [ ]:
# Required columns check
required = {"order_id","order_date","customer_id","product_id","category","city","qty","unit_price"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Missing columns in input: {missing}")

### Strip strings and fix types

In [ ]:
str_cols = ["category", "city"]
for c in str_cols:
    df[c] = df[c].astype("string").str.strip()

### Convert date, qty, unit_price

In [ ]:
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")
df["qty"] = pd.to_numeric(df["qty"], errors="coerce").astype("Int64")
df["unit_price"] = pd.to_numeric(df["unit_price"], errors="coerce")

### Basic data quality filters

In [ ]:
# - drop rows with invalid date/qty/unit_price
df = df.dropna(subset=["order_date", "qty", "unit_price"])
# - qty must be > 0, price >= 0
df = df[(df["qty"] > 0) & (df["unit_price"] >= 0)]

### Fill missing city with "Unknown"

In [ ]:
df["city"] = df["city"].fillna("Unknown")

### Compute total price

In [ ]:
df["total_price"] = df["qty"] * df["unit_price"]

### Deduplicate by (order_id, product_id) keeping last

In [ ]:
before_dedup = df.shape[0]
df = df.sort_values(["order_id", "product_id", "order_date"]).drop_duplicates(
    subset=["order_id", "product_id"], keep="last"
)

In [ ]:
after_dedup = df.shape[0]

In [ ]:
#  Add year-month for convenience
df["order_ym"] = df["order_date"].dt.to_period("M").astype(str)

In [ ]:
print(f" Cleaned rows: {after_dedup} (removed {before_dedup - after_dedup} duplicates)")

 Cleaned rows: 200 (removed 0 duplicates)


In [ ]:
#  Aggregate: daily revenue
daily_rev = (
    df.groupby(df["order_date"].dt.date, as_index=False)["total_price"]
      .sum()
      .rename(columns={"order_date":"order_date", "total_price":"daily_revenue"})
)

/tmp/ipython-input-4268930224.py:4: FutureWarning: A grouping was used that is not in the columns of the DataFrame and so was excluded from the result. This grouping will be included in a future version of pandas. Add the grouping as a column of the DataFrame to silence this warning.
  .sum()


## Load (SQLite)

In [ ]:
if DB_PATH.exists():
    DB_PATH.unlink()

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

# Create tables
cur.executescript("""
DROP TABLE IF EXISTS orders_clean;
DROP TABLE IF EXISTS daily_revenue;

CREATE TABLE orders_clean (
  order_id    INTEGER,
  order_date  TEXT,
  customer_id INTEGER,
  product_id  INTEGER,
  category    TEXT,
  city        TEXT,
  qty         INTEGER,
  unit_price  REAL,
  total_price REAL,
  order_ym    TEXT
);

CREATE TABLE daily_revenue (
  order_date    TEXT,
  daily_revenue REAL
);
""")
conn.commit()

In [ ]:
# Load dataframes into tables
df.to_sql("orders_clean", conn, if_exists="append", index=False)
daily_rev.to_sql("daily_revenue", conn, if_exists="append", index=False)
conn.commit()

print(f"Loaded into SQLite at: {DB_PATH}")

Loaded into SQLite at: /content/outputs/retail_etl.db


## Verify with quick SQL

In [ ]:
def q(sql):
    return pd.read_sql_query(sql, conn)

print("\nTop 5 cleaned rows:")
display(df.head())

print("\nTop 5 daily revenue:")
display(daily_rev.head())

print("\nSample checks:")
display(q("""
SELECT category, ROUND(SUM(total_price),2) AS revenue
FROM orders_clean
GROUP BY category
ORDER BY revenue DESC
LIMIT 5;
"""))

display(q("""
SELECT order_ym, ROUND(SUM(total_price),2) AS revenue
FROM orders_clean
GROUP BY order_ym
ORDER BY order_ym;
"""))

display(q("""
SELECT city, COUNT(*) AS orders
FROM orders_clean
GROUP BY city
ORDER BY orders DESC
LIMIT 5;
"""))

conn.close()





Top 5 cleaned rows:


,order_id,order_date,customer_id,product_id,category,city,qty,unit_price,total_price,order_ym
0,1001,2024-07-06,22,159,Clothing,Bengaluru,5,981.14,4905.7,2024-07
1,1002,2024-08-16,54,146,Stationery,Hyderabad,4,1534.79,6139.16,2024-08
2,1003,2024-08-09,30,119,Clothing,Chennai,3,1811.49,5434.47,2024-08
3,1004,2024-07-27,42,158,Grocery,Pune,5,1455.36,7276.8,2024-07
4,1005,2024-07-26,27,130,Stationery,Mumbai,1,1928.26,1928.26,2024-07



Top 5 daily revenue:


,daily_revenue
0,1754.91
1,7574.53
2,11473.04
3,9504.15
4,8955.77



Sample checks:


,category,revenue
0,Clothing,227951.82
1,Stationery,135966.77
2,Electronics,134669.62
3,Grocery,110084.30


,order_ym,revenue
0,2024-07,329465.62
1,2024-08,279206.89


,city,orders
0,Bengaluru,46
1,Chennai,45
2,Mumbai,39
3,Pune,37
4,Hyderabad,28


## Save artifacts

In [ ]:
clean_csv = OUTPUT_DIR / "clean_orders.csv"
daily_csv = OUTPUT_DIR / "daily_revenue.csv"
df.to_csv(clean_csv, index=False)
daily_rev.to_csv(daily_csv, index=False)

print("\n Outputs created:")
print(f"- {clean_csv}")
print(f"- {daily_csv}")
print(f"- {DB_PATH}")
print("\n ETL complete.")


 Outputs created:
- /content/outputs/clean_orders.csv
- /content/outputs/daily_revenue.csv
- /content/outputs/retail_etl.db

 ETL complete.


## Save project in drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#  destination folder
DEST = '/content/drive/MyDrive/Retail_ETL_Project'


import os
os.makedirs(DEST, exist_ok=True)

# Copy files
!cp /content/outputs/retail_etl.db "$DEST/"
!cp /content/outputs/clean_orders.csv "$DEST/"
!cp /content/outputs/daily_revenue.csv "$DEST/"

print(f" Files copied to: {DEST}")


 Files copied to: /content/drive/MyDrive/Retail_ETL_Project
